# Customer Balance Snapshot Query

This notebook connects to the Thrive Market DuckDB database and retrieves Thrive Cash balances for specified customers on a specific date.

## 1. Import Required Libraries

In [1]:
import duckdb
import pandas as pd

## 2. Connect to DuckDB Database

In [2]:
# Connect to the DuckDB database
db_path = './data/thrive.duckdb'
con = duckdb.connect(db_path)
print(f"Connected to {db_path}")

Connected to ./data/thrive.duckdb


## 3. Show staging model: stg_tc_data

In [3]:
query = """
select
    trans_id,
    trans_type,
    created_at,
    expired_at,
    customer_id,
    order_id,
    amount,
    reason
from stg_tc_data
"""
results = con.execute(query).fetch_df()
print(f"Query executed. Found {len(results)} record(s).")
display(results)

Query executed. Found 17 record(s).


,trans_id,trans_type,created_at,expired_at,customer_id,order_id,amount,reason
0,12011454,spent,2023-04-05 16:59:39,NaT,16161481,39061923.0,-16.07,None
1,11561486,earned,2023-03-08 23:06:16,2023-04-07 23:06:16,16161481,NaN,0.94,Refund
2,11561487,earned,2023-03-08 23:06:16,2023-04-07 23:06:16,16161481,NaN,0.90,Refund
3,11127421,earned,2023-02-05 13:26:14,2023-04-07 13:26:14,16161481,NaN,14.23,"Delivered, Not Received"
4,8900466,expired,2022-07-14 06:01:28,NaT,16161481,NaN,-5.00,None
5,8227592,earned,2022-05-14 03:10:07,2022-07-13 03:10:07,16161481,NaN,1.00,Damaged Item
6,8227590,earned,2022-05-14 03:10:06,2022-07-13 03:10:06,16161481,NaN,1.00,Damaged Item
7,8227587,earned,2022-05-14 03:10:05,2022-07-13 03:10:05,16161481,NaN,2.00,Short Expiry Product
8,8227588,earned,2022-05-14 03:10:05,2022-07-13 03:10:05,16161481,NaN,1.00,Short Expiry Product
9,8227471,spent,2022-05-14 02:27:40,NaT,16161481,32114336.0,-1.50,None


## 4. Show intermediate model: int_fifo_matched

In [4]:
query = """
select
    trans_id,
    trans_type,
    created_at,
    customer_id,
    amount,
    redeem_id
from int_fifo_matched
order by customer_id, created_at
"""
results = con.execute(query).fetch_df()
print(f"Query executed. Found {len(results)} record(s).")
display(results)

Query executed. Found 17 record(s).


,trans_id,trans_type,created_at,customer_id,amount,redeem_id
0,7781403,earned,2022-04-04 18:54:53,16161481,1.50,NaN
1,8227471,spent,2022-05-14 02:27:40,16161481,-1.50,7781403.0
2,8227587,earned,2022-05-14 03:10:05,16161481,2.00,NaN
3,8227588,earned,2022-05-14 03:10:05,16161481,1.00,NaN
4,8227590,earned,2022-05-14 03:10:06,16161481,1.00,NaN
5,8227592,earned,2022-05-14 03:10:07,16161481,1.00,NaN
6,8900466,expired,2022-07-14 06:01:28,16161481,-5.00,8227587.0
7,11127421,earned,2023-02-05 13:26:14,16161481,14.23,NaN
8,11561486,earned,2023-03-08 23:06:16,16161481,0.94,NaN
9,11561487,earned,2023-03-08 23:06:16,16161481,0.90,NaN


## 5. Show marts model: finance_report

In [5]:
query = """
select 
    customer_id,
    transaction_date,
    amount,
    cumulative_earned,
    cumulative_spent,
    cumulative_expired,
    current_balance
from finance_report
order by customer_id, transaction_date
"""
results = con.execute(query).fetch_df()
print(f"Query executed. Found {len(results)} record(s).")
display(results)

Query executed. Found 17 record(s).


,customer_id,transaction_date,amount,cumulative_earned,cumulative_spent,cumulative_expired,current_balance
0,16161481,2022-04-04 18:54:53,1.50,1.50,0.00,0.0,1.50
1,16161481,2022-05-14 02:27:40,-1.50,1.50,-1.50,0.0,3.00
2,16161481,2022-05-14 03:10:05,2.00,3.50,-1.50,0.0,5.00
3,16161481,2022-05-14 03:10:05,1.00,4.50,-1.50,0.0,6.00
4,16161481,2022-05-14 03:10:06,1.00,5.50,-1.50,0.0,7.00
5,16161481,2022-05-14 03:10:07,1.00,6.50,-1.50,0.0,8.00
6,16161481,2022-07-14 06:01:28,-5.00,6.50,-1.50,-5.0,13.00
7,16161481,2023-02-05 13:26:14,14.23,20.73,-1.50,-5.0,27.23
8,16161481,2023-03-08 23:06:16,0.94,21.67,-1.50,-5.0,28.17
9,16161481,2023-03-08 23:06:16,0.90,22.57,-1.50,-5.0,29.07


## 6. Execute SQL Query

In [6]:
# Query customer balance snapshots at a specific point in time
query = """
select 
    customer_id,
    transaction_date,
    current_balance as thrive_cash_balance
from finance_report
where customer_id in (23306353, 16161481)
  and transaction_date <= '2023-03-21'
--qualify row_number() over (partition by customer_id order by transaction_date desc, thrive_cash_balance asc) = 1
order by customer_id
"""

results = con.execute(query).fetch_df()
print(f"Query executed. Found {len(results)} record(s).")
display(results)

Query executed. Found 16 record(s).


,customer_id,transaction_date,thrive_cash_balance
0,16161481,2022-04-04 18:54:53,1.50
1,16161481,2022-05-14 02:27:40,3.00
2,16161481,2022-05-14 03:10:05,5.00
3,16161481,2022-05-14 03:10:05,6.00
4,16161481,2022-05-14 03:10:06,7.00
5,16161481,2022-05-14 03:10:07,8.00
6,16161481,2022-07-14 06:01:28,13.00
7,16161481,2023-02-05 13:26:14,27.23
8,16161481,2023-03-08 23:06:16,28.17
9,16161481,2023-03-08 23:06:16,29.07


In [7]:
# Summary statistics
if len(results) > 0:
    print("\nBalance Summary for 2023-03-21:")
    print("-" * 50)
    for idx, row in results.iterrows():
        print(f"Customer {row['customer_id']}: ${row['thrive_cash_balance']:.2f}")
else:
    print("No data found for the specified customers on 2023-03-21")


Balance Summary for 2023-03-21:
--------------------------------------------------
Customer 16161481: $1.50
Customer 16161481: $3.00
Customer 16161481: $5.00
Customer 16161481: $6.00
Customer 16161481: $7.00
Customer 16161481: $8.00
Customer 16161481: $13.00
Customer 16161481: $27.23
Customer 16161481: $28.17
Customer 16161481: $29.07
Customer 23306353: $40.00
Customer 23306353: $80.00
Customer 23306353: $110.00
Customer 23306353: $220.00
Customer 23306353: $240.00
Customer 23306353: $260.00


In [8]:
# Close the database connection
con.close()
print("Database connection closed.")

Database connection closed.
